<a href="https://colab.research.google.com/github/MrTyoom/Project-Jackal/blob/ml-experiments/Jackal_embedding_model_classifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Импорт датасета

In [ ]:
import pandas as pd

In [ ]:
df2 = pd.read_csv(r'first_command_new.csv')
df = pd.read_csv(r'commands-w2v-version.csv')

Переведем все текстовые интенты в лэйблы

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['lables'] = le.fit_transform(df.intent)
df2['lables'] = le.fit_transform(df2.intent)

In [ ]:
df[:1]

In [ ]:
df.lables

## 1. Создание эмбеддингов

Далее представлены различные модели для создания эмбеддингов и сразу же обучение модели по ним с помощью CatBoost

### Word2vec

Подготовка и реализация word2vec

In [ ]:
import gensim
import nltk
nltk.download('punkt')
from gensim.models import Word2Vec

In [ ]:
def generate_dictinoary_data(text):
    word_to_index= dict()
    index_to_word = dict()
    corpus = []
    count = 0
    vocab_size = 0
    
    for row in text:
        for word in row.split():
            word = word.lower()
            corpus.append(word)
            if word_to_index.get(word) == None:
                word_to_index.update ( {word : count})
                index_to_word.update ( {count : word })
                count  += 1
    vocab_size = len(word_to_index)
    length_of_corpus = len(corpus)
    
    return word_to_index,index_to_word,corpus,vocab_size,length_of_corpus


In [ ]:
#generate_dictinoary_data(df['command'])[0]

In [ ]:
# get the text data from the CSV file
text_data = df['command'].tolist()

# preprocess the text data
preprocessed_text_data = []
for text in text_data:
    text = text.lower()
    text = nltk.word_tokenize(text)
    preprocessed_text_data.append(text)

# train a Word2Vec model on the preprocessed text data
model = Word2Vec(preprocessed_text_data, vector_size = 100, window=5, min_count=1, workers=4)

In [ ]:
# use the Word2Vec model to get word embeddings
#embedding = model.wv['двигать']

In [ ]:
#embedding

Получили словарь слов из всех команд и вектор для каждого слова в словаре

### SBERT

Теперь сделаем эмбеддинги с помощью другой модели  — SBERT

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer as SBERT
sbert1 = SBERT('sberbank-ai/sbert_large_nlu_ru')


In [ ]:
EMBEDDING = sbert1.encode([s for s in df.command])
EMBEDDING2 = sbert1.encode([s for s in df2.command])

Получили эмбеддинги из SBERT'a

Получим модель из эмбеддингов с помощью Сatboost'a

In [ ]:
!pip install catboost

In [ ]:
import catboost
from catboost import * 

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['lables'] = le.fit_transform(df.intent)
df2['lables'] = le.fit_transform(df2.intent)

In [ ]:
X = df.drop("lables", axis=1)
y = df["lables"]

X2 = df2.drop("lables", axis=1)
y2 = df2["lables"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(EMBEDDING, y, test_size=0.33, random_state=42)

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(EMBEDDING2, y2, test_size=0.33, random_state=42)

In [ ]:
from catboost import CatBoostClassifier


model = CatBoostClassifier(iterations=100,
                         learning_rate=0.2,
                         depth=2, custom_loss=['AUC', 'Accuracy'])

model.fit(X_train, y_train,
         eval_set=(X_test, y_test),
          verbose=50, plot=True
)

print(f"Model is fitted: {str(model.is_fitted())}")
print(f"Model params: {model.get_params()}")


In [ ]:
from catboost import CatBoostClassifier


model = CatBoostClassifier(iterations=100,
                         learning_rate=0.2,
                         depth=2, custom_loss=['AUC', 'Accuracy'])

model.fit(X_train2, y_train2,
         eval_set=(X_test2, y_test2),
          verbose=50, plot=True
)

print(f"Model is fitted: {str(model.is_fitted())}")
print(f"Model params: {model.get_params()}")


Еще модели для сравнения

In [ ]:
model1 = CatBoostClassifier(
    learning_rate=0.7,
    iterations=100,
    random_seed=63,
    train_dir='learning_rate_0.7'
)

model2 = CatBoostClassifier(
    learning_rate=0.01,
    iterations=100,
    random_seed=63,
    train_dir='learning_rate_0.01'
)

model1.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    verbose=50
)

model2.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    verbose=50
)

In [ ]:
model3 = CatBoostClassifier(
    learning_rate=0.7,
    iterations=100,
    random_seed=63,
    train_dir='learning_rate_0.7'
)

model4 = CatBoostClassifier(
    learning_rate=0.01,
    iterations=100,
    random_seed=63,
    train_dir='learning_rate_0.01'
)

model3.fit(
    X_train2, y_train2,
    eval_set=(X_test2, y_test2),
    verbose=50
)

model4.fit(
    X_train2, y_train2,
    eval_set=(X_test2, y_test2),
    verbose=50
)

Теперь делаем predict

In [ ]:
y_pred = model2.predict(X_test, 
        prediction_type='Class', 
        ntree_start=0, 
        ntree_end=0, 
        thread_count=-1,
        verbose=None)

In [ ]:
y_pred2 = model4.predict(X_test2, 
        prediction_type='Class', 
        ntree_start=0, 
        ntree_end=0, 
        thread_count=-1,
        verbose=None)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred) 

Заметим, что классы: 0 — корабль влево, 1 — корабль вправо, 2 — ничего. Тогда пом матрице видно, что класс "ничего" никогда не определяется как лево/право (3я строчка матрицы), право было интерпретировано как ничего 2 раза, лево 1 (3й столбец матрицы.)

При этом, ошибочное определение "право" как лево было в 20 случаях, наоборот в 50. 

Делаем вывод, что классы не различаются между собой.

### BERT

In [ ]:
# загрузка библиотеки hugging face
!pip install transformers

#### cointegrated/rubert-tiny

In [ ]:
from transformers import AutoTokenizer, AutoModelForPreTraining

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")

model = AutoModelForPreTraining.from_pretrained("cointegrated/rubert-tiny")

Создаем эмбеддинги

In [ ]:
sentences = df.command

# Преобразование каждого предложения в эмбеддинг
embeddings_rubert_tiny = []
for sentence in sentences:
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = model(**inputs)
    last_hidden_state = outputs[0]  # замена строки
    sentence_embedding = last_hidden_state.mean(dim=1).squeeze().tolist()
    embeddings_rubert_tiny.append(sentence_embedding)

# получили список списков

Визуализация классов эмбеддингов

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

#  у нас есть эмбеддинги, хранящиеся в массиве embeddings, и метки классов, хранящиеся в массиве labels
# embeddings должны иметь форму (количество объектов, размерность эмбеддинга)

# задаем параметры t-SNE
tsne = TSNE(n_components=2, random_state=42)

# применяем t-SNE к эмбеддингам
embeddings_rubert_tiny_tsne = tsne.fit_transform(embeddings_rubert_tiny)

# задаем цвета для каждого класса
color_dict = {0: 'red', 1: 'blue', 2: 'green'} # 


# создаем график
fig, ax = plt.subplots(figsize=(10, 10))

# для каждого класса рисуем точки соответствующего цвета
for label in np.unique(df.lables):
    mask = df.lables == label
    ax.scatter(embeddings_rubert_tiny_tsne[mask, 0], embeddings_rubert_tiny_tsne[mask, 1], c=color_dict[label], label=label, alpha=0.7, s=100)

ax.legend()
ax.set_title('t-SNE visualization of embeddings')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Ваш список списков эмбеддингов и список лэйблов
embeddings = embeddings_rubert_tiny
labels = df.lables

# Создание 3D-графика
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')


# Извлечение координат x, y, z из списка списков эмбеддингов
x = [embedding[0] for embedding in embeddings]
y = [embedding[1] for embedding in embeddings]
z = [embedding[2] for embedding in embeddings]

# Определение цвета точек на основании лейблов
color_dict = {0: 'r', 1: 'b', 2: 'g'}
colors = [color_dict[label] for label in labels]

# Нанесение точек на график и задание цветов
ax.scatter(x, y, z, c=colors)

# Добавление легенды
for label, color in color_dict.items():
    ax.scatter([], [], [], c=color, label=label)
ax.legend()

# Отображение графика
plt.show()


In [ ]:
import plotly.graph_objs as go

# Ваш список списков эмбеддингов и список лэйблов
embeddings = embeddings_rubert_tiny
labels = df.lables

# Извлечение координат x, y, z из списка списков эмбеддингов
x = [embedding[0] for embedding in embeddings]
y = [embedding[1] for embedding in embeddings]
z = [embedding[2] for embedding in embeddings]

# Определение цвета точек на основании лейблов
color_dict = {0: 'red', 1: 'blue', 2: 'green'}
colors = [color_dict[label] for label in labels]

# Создание трехмерного графика
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(color=colors, size=3))])

# Добавление подписей к осям
fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'))

# Включение интерактивности и возможности вращения по всем осям
fig.update_layout(scene_camera=dict(up=dict(x=0, y=0, z=1), eye=dict(x=0, y=1, z=1)))

# Отображение графика
fig.show()


Разделим датасет на тестовую, валидационную и тренировочную выборки

In [ ]:
from sklearn.model_selection import train_test_split

X = embeddings_rubert_tiny # матрица признаков
y = df.lables # вектор целевых переменных

# Сначала разбиваем на train и test, с перемешиванием
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

# Далее разбиваем train на train и validation, с перемешиванием
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True, random_state=42)

Найдем оптимальные параметры с помощью Optuna

In [ ]:
!pip install Optuna

In [ ]:
!pip install catboost

In [ ]:
import optuna
import catboost as cb
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, make_scorer, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import classification_report

In [ ]:
# Определяем функцию для оптимизации гиперпараметров
def objective(trial):
    params = {
        'iterations': 1000,
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'depth': trial.suggest_int('depth', 2, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-5, 10, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.01, 100.0, log=True),
        'random_strength': trial.suggest_float('random_strength', 1e-5, 10, log=True),
        'border_count': trial.suggest_int('border_count', 5, 255),
        'eval_metric': 'Accuracy',
        'random_seed': 42,
        'use_best_model': True,
        'verbose': False,
    }
    
    
    # Создаем модель CatBoost и обучаем ее
    model = CatBoostClassifier(**params, loss_function='MultiClass')   
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=50, verbose=False)

    # Вычисляем F1-меру на тестовой выборке
    y_pred = model.predict(X_test)
    score = f1_score(y_test, y_pred, average='macro')
    
    return score

# Запускаем оптимизацию гиперпараметров с помощью Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Получаем лучшие параметры и обучаем модель на всех данных
best_params = study.best_params

# # Оцениваем качество модели на тестовой выборке
# y_pred = model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Test accuracy: {accuracy:.3f}')

print('Best hyperparameters: ', study.best_params)
print('Best accuracy: ', study.best_value)


Вывоl optuna для модели выше: 


Построим модель по этим параметрам: 

In [ ]:
# Создаем модель с найденными гиперпараметрами # ЕЩЕ НЕ ПОДСТАВИЛА
model_rubert_tiny = CatBoostClassifier(
    learning_rate=0.06539173226910043,
    depth=3,
    l2_leaf_reg=2.4173397465978706e-05,
    bagging_temperature=0.11366113651396821,
    random_strength=0.0030236222243492385,
    border_count=85,
    eval_metric='Accuracy',
    random_seed=42,
    use_best_model=True,
    verbose=False,
)

# Обучаем модель на тренировочной выборке
model_rubert_tiny.fit(X_train, y_train, eval_set=(X_val, y_val))

# Получаем предсказания на тестовой выборке
y_pred_rubert_tiny = model_rubert_base.predict(X_test)

# Оцениваем качество модели
print(classification_report(y_test, y_pred))


#### cointegrated/rubert-tiny2

In [ ]:
from transformers import AutoTokenizer, AutoModelForPreTraining

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")

model = AutoModelForPreTraining.from_pretrained("cointegrated/rubert-tiny2")

Создаем эмбеддинги

In [ ]:
sentences = df.command

# Преобразование каждого предложения в эмбеддинг
embeddings_rubert_tiny = []
for sentence in sentences:
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = model(**inputs)
    last_hidden_state = outputs[0]  # замена строки
    sentence_embedding = last_hidden_state.mean(dim=1).squeeze().tolist()
    embeddings_rubert_tiny.append(sentence_embedding)

# получили список списков

Визуализация классов эмбеддингов

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

#  у нас есть эмбеддинги, хранящиеся в массиве embeddings, и метки классов, хранящиеся в массиве labels
# embeddings должны иметь форму (количество объектов, размерность эмбеддинга)

# задаем параметры t-SNE
tsne = TSNE(n_components=2, random_state=42)

# применяем t-SNE к эмбеддингам
embeddings_rubert_tiny_tsne = tsne.fit_transform(embeddings_rubert_tiny)

# задаем цвета для каждого класса
color_dict = {0: 'red', 1: 'blue', 2: 'green'} # 


# создаем график
fig, ax = plt.subplots(figsize=(10, 10))

# для каждого класса рисуем точки соответствующего цвета
for label in np.unique(df.lables):
    mask = df.lables == label
    ax.scatter(embeddings_rubert_tiny_tsne[mask, 0], embeddings_rubert_tiny_tsne[mask, 1], c=color_dict[label], label=label, alpha=0.7, s=100)

ax.legend()
ax.set_title('t-SNE visualization of embeddings')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Ваш список списков эмбеддингов и список лэйблов
embeddings = embeddings_rubert_tiny
labels = df.lables

# Создание 3D-графика
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')


# Извлечение координат x, y, z из списка списков эмбеддингов
x = [embedding[0] for embedding in embeddings]
y = [embedding[1] for embedding in embeddings]
z = [embedding[2] for embedding in embeddings]

# Определение цвета точек на основании лейблов
color_dict = {0: 'r', 1: 'b', 2: 'g'}
colors = [color_dict[label] for label in labels]

# Нанесение точек на график и задание цветов
ax.scatter(x, y, z, c=colors)

# Добавление легенды
for label, color in color_dict.items():
    ax.scatter([], [], [], c=color, label=label)
ax.legend()

# Отображение графика
plt.show()


In [ ]:
import plotly.graph_objs as go

# Ваш список списков эмбеддингов и список лэйблов
embeddings = embeddings_rubert_tiny
labels = df.lables

# Извлечение координат x, y, z из списка списков эмбеддингов
x = [embedding[0] for embedding in embeddings]
y = [embedding[1] for embedding in embeddings]
z = [embedding[2] for embedding in embeddings]

# Определение цвета точек на основании лейблов
color_dict = {0: 'red', 1: 'blue', 2: 'green'}
colors = [color_dict[label] for label in labels]

# Создание трехмерного графика
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(color=colors, size=3))])

# Добавление подписей к осям
fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'))

# Включение интерактивности и возможности вращения по всем осям
fig.update_layout(scene_camera=dict(up=dict(x=0, y=0, z=1), eye=dict(x=0, y=1, z=1)))

# Отображение графика
fig.show()


Разделим датасет на тестовую, валидационную и тренировочную выборки

In [ ]:
from sklearn.model_selection import train_test_split

X = embeddings_rubert_tiny # матрица признаков
y = df.lables # вектор целевых переменных

# Сначала разбиваем на train и test, с перемешиванием
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

# Далее разбиваем train на train и validation, с перемешиванием
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True, random_state=42)

Найдем оптимальные параметры с помощью Optuna

In [ ]:
# Определяем функцию для оптимизации гиперпараметров
def objective(trial):
    params = {
        'iterations': 1000,
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'depth': trial.suggest_int('depth', 2, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-5, 10, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.01, 100.0, log=True),
        'random_strength': trial.suggest_float('random_strength', 1e-5, 10, log=True),
        'border_count': trial.suggest_int('border_count', 5, 255),
        'eval_metric': 'Accuracy',
        'random_seed': 42,
        'use_best_model': True,
        'verbose': False,
    }
    
    
    # Создаем модель CatBoost и обучаем ее
    model = CatBoostClassifier(**params, loss_function='MultiClass')   
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=50, verbose=False)

    # Вычисляем F1-меру на тестовой выборке
    y_pred = model.predict(X_test)
    score = f1_score(y_test, y_pred, average='macro')
    
    return score

# Запускаем оптимизацию гиперпараметров с помощью Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Получаем лучшие параметры и обучаем модель на всех данных
best_params = study.best_params

# # Оцениваем качество модели на тестовой выборке
# y_pred = model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Test accuracy: {accuracy:.3f}')

print('Best hyperparameters: ', study.best_params)
print('Best accuracy: ', study.best_value)


Вывоl optuna для модели выше: 


Построим модель по этим параметрам: 

In [ ]:
# Создаем модель с найденными гиперпараметрами //ЕЩЕ НЕ ПОДСТАВИЛА
model_rubert_tiny = CatBoostClassifier(
    learning_rate=0.06539173226910043,
    depth=3,
    l2_leaf_reg=2.4173397465978706e-05,
    bagging_temperature=0.11366113651396821,
    random_strength=0.0030236222243492385,
    border_count=85,
    eval_metric='Accuracy',
    random_seed=42,
    use_best_model=True,
    verbose=False,
)

# Обучаем модель на тренировочной выборке
model_rubert_tiny.fit(X_train, y_train, eval_set=(X_val, y_val))

# Получаем предсказания на тестовой выборке
y_pred_rubert_tiny = model_rubert_base.predict(X_test)

# Оцениваем качество модели
print(classification_report(y_test, y_pred))


#### DeepPavlov/rubert-base-cased

In [ ]:
from transformers import AutoTokenizer, AutoModel

# Загрузка токенизатора и модели
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased")


In [ ]:
sentences = df.command

# Преобразование каждого предложения в эмбеддинг
embeddings_rubert_base = []
for sentence in sentences:
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = model(**inputs)
    last_hidden_state = outputs.last_hidden_state
    sentence_embedding = last_hidden_state.mean(dim=1).squeeze().tolist()
    embeddings_rubert_base.append(sentence_embedding)

# получили список списков

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

#  у нас есть эмбеддинги, хранящиеся в массиве embeddings, и метки классов, хранящиеся в массиве labels
# embeddings должны иметь форму (количество объектов, размерность эмбеддинга)

# задаем параметры t-SNE
tsne = TSNE(n_components=2, random_state=42)

# применяем t-SNE к эмбеддингам
embeddings_rubert_case_tsne = tsne.fit_transform(embeddings_rubert_base)

# задаем цвета для каждого класса
color_dict = {0: 'red', 1: 'blue', 2: 'green'} # 


# создаем график
fig, ax = plt.subplots(figsize=(10, 10))

# для каждого класса рисуем точки соответствующего цвета
for label in np.unique(df.lables):
    mask = df.lables == label
    ax.scatter(embeddings_rubert_case_tsne[mask, 0], embeddings_rubert_case_tsne[mask, 1], c=color_dict[label], label=label, alpha=0.7, s=100)

ax.legend()
ax.set_title('t-SNE visualization of embeddings')
plt.show()


3д график с кручением ползунком:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Ваш список списков эмбеддингов и список лэйблов
embeddings = embeddings_rubert_base
labels = df.lables

# Создание 3D-графика
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')


# Извлечение координат x, y, z из списка списков эмбеддингов
x = [embedding[0] for embedding in embeddings]
y = [embedding[1] for embedding in embeddings]
z = [embedding[2] for embedding in embeddings]

# Определение цвета точек на основании лейблов
color_dict = {0: 'r', 1: 'b', 2: 'g'}
colors = [color_dict[label] for label in labels]

# Нанесение точек на график и задание цветов
ax.scatter(x, y, z, c=colors)

# Добавление легенды
for label, color in color_dict.items():
    ax.scatter([], [], [], c=color, label=label)
ax.legend()

# Отображение графика
plt.show()


3д график с кручением мышкой:

In [ ]:
import plotly.graph_objs as go

# Ваш список списков эмбеддингов и список лэйблов
embeddings = embeddings_rubert_base
labels = df.lables

# Извлечение координат x, y, z из списка списков эмбеддингов
x = [embedding[0] for embedding in embeddings]
y = [embedding[1] for embedding in embeddings]
z = [embedding[2] for embedding in embeddings]

# Определение цвета точек на основании лейблов
color_dict = {0: 'red', 1: 'blue', 2: 'green'}
colors = [color_dict[label] for label in labels]

# Создание трехмерного графика
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(color=colors, size=3))])

# Добавление подписей к осям
fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'))

# Включение интерактивности и возможности вращения по всем осям
fig.update_layout(scene_camera=dict(up=dict(x=0, y=0, z=1), eye=dict(x=0, y=1, z=1)))

# Отображение графика
fig.show()


In [ ]:
from sklearn.model_selection import train_test_split

X = embeddings_rubert_base # матрица признаков
y = df.lables # вектор целевых переменных

# Сначала разбиваем на train и test, с перемешиванием
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

# Далее разбиваем train на train и validation, с перемешиванием
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
import optuna
import catboost as cb
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, make_scorer, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import classification_report

In [ ]:
# Определяем функцию для оптимизации гиперпараметров
def objective(trial):
    params = {
        'iterations': 1000,
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'depth': trial.suggest_int('depth', 2, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-5, 10, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.01, 100.0, log=True),
        'random_strength': trial.suggest_float('random_strength', 1e-5, 10, log=True),
        'border_count': trial.suggest_int('border_count', 5, 255),
        'eval_metric': 'Accuracy',
        'random_seed': 42,
        'use_best_model': True,
        'verbose': False,
    }
    
    
    # Создаем модель CatBoost и обучаем ее
    model = CatBoostClassifier(**params, loss_function='MultiClass')   
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=50, verbose=False)

    # Вычисляем F1-меру на тестовой выборке
    y_pred = model.predict(X_test)
    score = f1_score(y_test, y_pred, average='macro')
    
    return score

# Запускаем оптимизацию гиперпараметров с помощью Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Получаем лучшие параметры и обучаем модель на всех данных
best_params = study.best_params

# # Оцениваем качество модели на тестовой выборке
# y_pred = model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Test accuracy: {accuracy:.3f}')

print('Best hyperparameters: ', study.best_params)
print('Best accuracy: ', study.best_value)


Вывоl optuna для модели выше: 
"Best hyperparameters:  {'learning_rate': 0.06539173226910043, 'depth': 3, 'l2_leaf_reg': 2.4173397465978706e-05, 'bagging_temperature': 0.11366113651396821, 'random_strength': 0.0030236222243492385, 'border_count': 85}
Best accuracy:  0.8583546244251407"

Построим модель по этим параметрам: 

In [ ]:
# Создаем модель с найденными гиперпараметрами
model_rubert_base = CatBoostClassifier(
    learning_rate=0.06539173226910043,
    depth=3,
    l2_leaf_reg=2.4173397465978706e-05,
    bagging_temperature=0.11366113651396821,
    random_strength=0.0030236222243492385,
    border_count=85,
    eval_metric='Accuracy',
    random_seed=42,
    use_best_model=True,
    verbose=False,
)

# Обучаем модель на тренировочной выборке
model_rubert_base.fit(X_train, y_train, eval_set=(X_val, y_val))

# Получаем предсказания на тестовой выборке
y_pred_rubert_case = model_rubert_base.predict(X_test)

# Оцениваем качество модели
print(classification_report(y_test, y_pred))


## 2. Обучение моделей на основе полученных эмбеддингов